In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH


In [2]:
# 라이브러리 선언
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.document_compressors import EmbeddingsFilter
import re
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


In [ ]:
# 임베딩 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# DB 경로 설정
persist_directory = "../.db/faiss"

In [3]:
def pretty_print(query, result):
    print(f"'{query}'와 유사한 상품 검색 결과:")
    print("-" * 60)

    if result:
        for i, (doc) in enumerate(result, 1):
            print(f"[결과 {i}]")
            print(f"{doc.page_content}")
            # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
            try:
                print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
                print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
                print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
                print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
                print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
                print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
                print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
                print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
                print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
                print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
                print(f"- 주요 특징 및 기능:")
                feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
                feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
                for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                    print(f"  - {title} : {value}")
                    if i < 3: break
                print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
            except Exception as e:
                print(f"메타데이터 처리 중 오류 발생: {str(e)}")
            print("-" * 60)
    else:
        print("검색 결과가 없습니다.")
        print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")

In [4]:
# 의도 기반 필터링 함수 선언

def intent_based_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격필터
    filter_dict.update(intent_based_price_filtering(query))

    # 브랜드필터

    # 속성필터

    return filter_dict

def intent_based_price_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict


In [12]:
def extract_documents_from_faiss(faiss_db):
    """FAISS DB에서 Document 객체들을 추출"""
    documents = []
    
    # docstore에서 모든 Document 객체 추출
    for doc_id in faiss_db.docstore._dict.keys():
        doc = faiss_db.docstore.search(doc_id)
        # 이미 Document 객체이므로 그대로 사용
        documents.append(doc)
    
    return documents

In [27]:
# faiss retriever 초기화합니다.
faiss_vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)



In [28]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# bm25 retriever를 초기화합니다.
documents = extract_documents_from_faiss(faiss_vectorstore)
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

In [15]:
# 검색 결과 문서를 가져옵니다.
query = "100만원 이상 삼성전자 TV"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
print_result(query, ensemble_result)
print()

print("[BM25 Retriever]")
print_result(query, bm25_result)
print()

print("[FAISS Retriever]")
print_result(query, faiss_result)
print()

[Ensemble Retriever]
'100만원 이상 삼성전자 TV'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
삼성전자의 [최상급] 삼성전자 OLED TV KQ77SC95AFXKR. 카테고리: TV·영상가전 > TV > 초대형(190cm~). 상품 상태: 진열상품. 가격 정보: 판매가 3,580,000원, 할인가 3,288,300원. 가전제품입니다. 해시태그: #OLED#OLEDTV#OLEDTV#77인치#삼성TV#삼성#TV#티비#올레드#전시상품.
- 상품상태: 진열상품
- 브랜드명: 삼성전자
- 상품명: [최상급] 삼성전자 OLED TV KQ77SC95AFXKR
- 품목정보: TV>OLED TV
- 카테고리: TV·영상가전>TV>초대형(190cm~)
- 판매가: 3,580,000원
- 할인가: 3,288,300원
- 최대혜택가: 3,288,300원
- 카드할인율: 0%
- 할인카드: 
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0021843014
------------------------------------------------------------
[결과 2]
삼성전자의 LED TV, LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]. 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 1,179,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 1,096,470원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]
- 품목정보: TV>LED TV
- 카테고리: TV·영상가전>TV>대형(~189cm)
- 판매가: 1,179,000원
- 할인가: 1,179,000원
- 최대

In [26]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever]
).configurable_fields(
    weights=ConfigurableField(
        id="ensemble_weights",
        name="Search Weights",
        description="BM25 vs FAISS 가중치"
    ),
    search_kwargs=ConfigurableField(
        id="search_params", 
        name="Search Parameters",
        description="검색 파라미터 (k값 등)"
    )
)


ValueError: Configuration key search_kwargs not found in retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001373D64ACD0>, k=2), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001373182CC90>, search_kwargs={'k': 2})] weights=[0.5, 0.5]: available keys are dict_keys(['name', 'tags', 'metadata', 'retrievers', 'weights', 'c', 'id_key'])

In [25]:
from langchain_core.runnables import ConfigurableField

ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights",
    )
)

ValueError: Configuration key search_kwargs not found in retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001373D64ACD0>, k=2), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001373182CC90>, search_kwargs={'k': 2})] weights=[0.5, 0.5]: available keys are dict_keys(['name', 'tags', 'metadata', 'retrievers', 'weights', 'c', 'id_key'])

In [16]:

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "100만원 이상 삼성전자 TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

# 필터링 옵션 입력 받기
filter_dict = intent_based_filtering(query)
# filter_dict = {"GOODS_NO": '0031055945'}
# filter_dict = {
#     "SALE_PRC": {
#         "$gte": 1000000,  # 최소 가격
#         "$lte": 2000000   # 최대 가격
#     }
# }

# 검색어에 따라 적절한 설정을 지정
config = {
    "configurable": {
        "search_type": "similarity_score_threshold", # similarity, similarity_score_threshold, mmr
        "search_kwargs": {
            "score_threshold": 0.4,
            # "k": 4,
            "filter": filter_dict
        },
        "ensemble_weights": [0.7, 0.3]
    }
}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
result = ensemble_retriever.invoke(query, config=config)

print(f"필터: {filter_dict}")

print_result(query, result)

필터: {'SALE_PRC': {'$gte': 1000000}}
'100만원 이상 삼성전자 TV'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
삼성전자의 [최상급] 삼성전자 OLED TV KQ77SC95AFXKR. 카테고리: TV·영상가전 > TV > 초대형(190cm~). 상품 상태: 진열상품. 가격 정보: 판매가 3,580,000원, 할인가 3,288,300원. 가전제품입니다. 해시태그: #OLED#OLEDTV#OLEDTV#77인치#삼성TV#삼성#TV#티비#올레드#전시상품.
- 상품상태: 진열상품
- 브랜드명: 삼성전자
- 상품명: [최상급] 삼성전자 OLED TV KQ77SC95AFXKR
- 품목정보: TV>OLED TV
- 카테고리: TV·영상가전>TV>초대형(190cm~)
- 판매가: 3,580,000원
- 할인가: 3,288,300원
- 최대혜택가: 3,288,300원
- 카드할인율: 0%
- 할인카드: 
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0021843014
------------------------------------------------------------
[결과 2]
삼성전자의 LED TV, LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]. 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 1,179,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 1,096,470원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]
- 품목정보: TV>LED TV
- 카테고리: TV·영상가전>TV>대형(~189cm)
- 판매가: 1,179,000원
- 할인가: 

# 앙상블 검색기(Ensemble Retriever)

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

**주요 특징**
1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. 결과 재순위화: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. 하이브리드 검색: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

**장점**
- Sparse retriever: 키워드 기반 검색에 효과적
- Dense retriever: 의미적 유사성 기반 검색에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.
